In [1]:
import torch
from models import GKT
from processing import load_dataset, pad_collate
from utils import build_dense_graph

# GPU 사용 여부
device = torch.device("cpu")
print("사용 중인 디바이스:", device)


사용 중인 디바이스: cpu


In [2]:
# 그래프 및 데이터셋 초기화
concept_num = 1815  # 문제(개념) 개수
hidden_dim = 32
embedding_dim = 32
edge_type_num = 2
graph_type = "Dense"

# Dense 그래프 생성
graph = build_dense_graph(concept_num).to(device)

# 모델 가중치 로드
model_path = "logs\expGKT-Dense2024-12-20 11-14-06\GKT-Dense.pt"  # 사전 학습된 모델 경로

In [11]:
import pandas as pd

# 특정 UserID를 위한 데이터 로드
file_path = "data\\filtered_combined_user_data.csv"  # 데이터 파일 경로
user_id = "1a851549-0ce5-43da-b73c-ffff1117f116"  # 예측할 UserID

# 데이터프레임에서 유저 데이터 로드
df = pd.read_csv(file_path)
user_data = df[df['UserID'] == user_id]

# Step 0 - 정렬: 가장 오래된 기록부터 정렬
user_data.sort_values(by=["UserID", "CreDate"], inplace=True)  # "CreDate" 컬럼을 기준으로 정렬

# Step 2 - Enumerate skill id
user_data['skill'], _ = pd.factorize(user_data['QuizCode'], sort=True)  # we can also use problem_id to represent exercises

# correct 생성 (O -> 1, X -> 0)
user_data['Correct'] = user_data['Correct'].map({'O': 1, 'X': 0})

# Step 3 - Cross skill id with answer to form a synthetic feature
# use_binary: (0,1); !use_binary: (1,2,3,4,5,6,7,8,9,10,11,12). Either way, the correct result index is guaranteed to be 1
user_data['skill_with_answer'] = user_data['skill'] * 2 + user_data['Correct']

print(user_data.tail(10))

user_data = user_data.iloc[:-10]

# 유저 풀이 시퀀스 및 다음 문제 정의
features = user_data['skill_with_answer'].tolist()
questions = user_data['skill'].tolist()


                                        UserID         mCode  No  QuizCode  \
19751544  1a851549-0ce5-43da-b73c-ffff1117f116  T1ME12U03011   6  30161292   
19751545  1a851549-0ce5-43da-b73c-ffff1117f116  T1ME12U03011   7  30161293   
19751546  1a851549-0ce5-43da-b73c-ffff1117f116  T1ME12U03011   8  30161294   
19751547  1a851549-0ce5-43da-b73c-ffff1117f116  T1ME12U03011   9  30161295   
17897686  1a851549-0ce5-43da-b73c-ffff1117f116  T1ME12U03016   1  30161254   
17897687  1a851549-0ce5-43da-b73c-ffff1117f116  T1ME12U03016   2  30161253   
17897688  1a851549-0ce5-43da-b73c-ffff1117f116  T1ME12U03016   3  30161252   
19751548  1a851549-0ce5-43da-b73c-ffff1117f116  T1ME12U03016   1  30161254   
19751549  1a851549-0ce5-43da-b73c-ffff1117f116  T1ME12U03016   2  30161253   
19751550  1a851549-0ce5-43da-b73c-ffff1117f116  T1ME12U03016   3  30161252   

                     Answer  Correct                  CreDate  LM_IDX  skill  \
19751544                 10        1  2024-09-26 19:04:00.000

C:\Users\user\AppData\Local\Temp\ipykernel_4536\809071246.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_data.sort_values(by=["UserID", "CreDate"], inplace=True)  # "CreDate" 컬럼을 기준으로 정렬
C:\Users\user\AppData\Local\Temp\ipykernel_4536\809071246.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_data['skill'], _ = pd.factorize(user_data['QuizCode'], sort=True)  # we can also use problem_id to represent exercises
C:\Users\user\AppData\Local\Temp\ipykernel_4536\809071246.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sli

In [12]:
next_skills = [456, 457, 458, 459, 444, 443, 442, 444, 443, 442]  # 다음에 풀 문제들

In [ ]:
# 디바이스 설정 (GPU 또는 CPU)
device = torch.device("cpu")
print("사용 중인 디바이스:", device)

# 모델 초기화
model = GKT(concept_num, hidden_dim, embedding_dim, edge_type_num, graph_type, graph=graph).to(device)

# 데이터 텐서로 변환 및 디바이스 이동
features_tensor = torch.tensor(features, dtype=torch.long).unsqueeze(0).to(device)
questions_tensor = torch.tensor(questions, dtype=torch.long).unsqueeze(0).to(device)
next_skills_tensor = torch.tensor(next_skills, dtype=torch.long).to(device)

# Dense 그래프도 동일한 디바이스로 이동
graph = graph.to(device)

# 모델 가중치 로드
model.load_state_dict(torch.load(model_path, map_location=device))
model.eval()

# 모델 예측 수행
with torch.no_grad():
    pred_res, _, _, _ = model(features_tensor, questions_tensor)  # 입력값과 동일한 디바이스에서 수행
    next_preds = pred_res.squeeze(0)[-len(next_skills):]  # 다음 문제에 해당하는 예측값

threshold = 0.8318

# 결과 출력
print("다음에 풀 10문제의 예측값 및 정오표:")
for i, pred in enumerate(next_preds.tolist()):
    result = "1" if pred >= threshold else "0"
    print(f"문제 {next_skills[i]}: 정답 확률 = {pred:.4f} ({result}) 원래 정답 : {user_data['Correct']}")

사용 중인 디바이스: cpu


C:\Users\user\AppData\Local\Temp\ipykernel_4536\2638515844.py:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=dev

다음에 풀 10문제의 예측값 및 정오표:


KeyError: 0